In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd
import numpy as np
import csv
import utils.baseline_functions as base

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning


In [2]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/data/ky-data/kentucky_data.csv").sort_values('person_id')
x = data.loc[:,:'current_violence20'].drop(['arnold_fta_raw', 'arnold_nca_raw', 'arnold_nvca_raw'], axis=1)
y = data['general_two_year'].values

In [5]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(X=x,
                                 Y=y,
                                 C=c,
                                 seed=816)

#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(X=x,
                           Y=y, 
                           C=c,
                           seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(X=x,
                             Y=y,
                             C=c,
                             seed=816)

#### Random Forest
n_estimators =  [100,150,200]
depth = [7,8,9]
rf_summary = base.RF(X=x,
                     Y=y,
                     depth=depth, 
                     estimators=n_estimators, 
                     seed=816)

#### XGBoost
learning_rate = [0.1]
depth = [4,5,6]
n_estimators = [100,150]
xgb_summary = base.XGB(X=x,
                       Y=y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators, 
                       seed=816)

#### save results
summary_general_2_KY = {"Logistic": logistic_summary,
                        "Lasso": lasso_summary,
                        "LinearSVM": svm_summary,
                        "RF": rf_summary,
                        "XGBoost": xgb_summary}

C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scalars
  prob = labels.sum() / len(labels)
C:\Users\binha\Documents\Duke\Cynthia Research\interpretable-machine-learning\utils\fairness_functions.py:89: RuntimeWarning: invalid value encountered in longlong_scal

In [6]:
results,auc = [],[]
for model_name, model_summary in summary_general_2_KY.items():
    results.append([model_name, 
                    np.mean(model_summary['holdout_test_auc']), 
                    np.mean(model_summary['auc_diffs'])])
    auc.append(np.mean(model_summary['holdout_test_auc']))
results

[['Logistic', 0.7448501156009045, 0.00033773769036435917],
 ['Lasso', 0.7448547181826648, 0.0003174946709891824],
 ['LinearSVM', 0.7458525142754369, 0.00027971450211032866],
 ['RF', 0.7532345647409798, 0.009182017226175555],
 ['XGBoost', 0.7585086965543292, 0.012445084700868182]]

In [7]:
path = "./kentucky/logs/baselines/"
results = [["", "Logistic", "Lasso", "Linear SVM", "Random Forest", "XGBoost", "Performance Range"],
           ["General", np.str((round(np.mean(logistic_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(logistic_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(lasso_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(lasso_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(svm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(svm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(rf_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(rf_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(xgb_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(xgb_summary['holdout_test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3)]]
with open(path + 'KY-two-year-baseline-summary.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)